In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [3]:
import pandas as pd

In [4]:
df = pd.read_pickle("dataset/test_500.pkl", compression="zip")

In [5]:
df.head()

,x,y,z,vx,vy,vz,px,py,pz,distance,speed,label
0,-117.0,87.0,-33.0,0.47,0.03,0.03,1.90,-0.53,0.57,150.0,5.232,3
1,-117.0,87.0,-33.0,0.47,0.03,0.03,1.90,-0.53,0.57,150.0,5.232,50691
2,-113.0,87.0,-33.0,0.47,0.04,0.03,2.00,-0.34,0.76,146.0,5.232,4
3,-113.0,87.0,-33.0,0.47,0.04,0.03,2.00,-0.34,0.76,146.0,5.232,50692
4,-109.0,87.0,-33.0,0.47,0.04,0.02,2.16,-0.05,0.95,143.0,5.232,5


In [23]:
labels = df["label"].unique()
type(labels), labels.shape

(numpy.ndarray, (1000,))

In [49]:
# df.to_numpy()[:,11:]

array([[3.0000e+00],
       [5.0691e+04],
       [4.0000e+00],
       ...,
       [5.1189e+04],
       [5.0200e+02],
       [5.1190e+04]])

In [187]:
data = df.to_numpy()[:,:11]
labels = df.to_numpy()[:,11:]
data.shape, labels.shape

# data = df.to_numpy()

((6000, 11), (6000, 1))

In [188]:
num_epochs = 50
batch_size = 32
class_labels = df["label"].unique()
num_classes = len(class_labels)

In [189]:
class MyDataset(Dataset):
    def __init__(self, data,labels):
        # self.data = torch.tensor(data).float()
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.int64)
        # self.labels = torch.tensor(data).float()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx] , self.labels[idx]

In [148]:
# from torch.utils.data import TensorDataset, DataLoader
# Create instances of the dataset
# dataset = MyDataset(data)
#
# # Create a dataloader with a batch size of 32 and shuffle the data
# data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# dataset = TensorDataset(torch.tensor(data, dtype=torch.float32), torch.tensor(labels, dtype=torch.int64))

In [149]:
data.shape

(6000, 12)

In [190]:
# from torch.utils.data import Dataset, DataLoader
#
# class MyDataset(Dataset):
#     def __init__(self, data, labels):
#         self.data = data
#         self.labels = labels
#
#     def __len__(self):
#         return len(self.data)
#
#     def __getitem__(self, idx):
#         return self.data[idx], self.labels[idx]

# Create instances of the dataset
dataset = MyDataset(data, labels)

# Create a dataloader with a batch size of 32 and shuffle the data
train_data = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [191]:
# Define the generator and discriminator networks
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.layer1 = nn.Linear(12, 64)  # added 1 more input for the label
        self.layer2 = nn.Linear(64, 128)
        self.layer3 = nn.Linear(128, 256)
        self.layer4 = nn.Linear(256, 512)
        self.output = nn.Linear(512, 11)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = torch.relu(self.layer4(x))
        x = self.output(x)
        return x

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.layer1 = nn.Linear(11, 64)
        self.layer2 = nn.Linear(64, 128)
        self.layer3 = nn.Linear(128, 256)
        self.output = nn.Linear(256, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = torch.sigmoid(self.output(x))
        return x

In [192]:
# Create instances of the generator and discriminator
generator = Generator()
discriminator = Discriminator()

In [193]:
criterion = nn.BCELoss()
gen_optimizer = optim.Adam(generator.parameters(), lr=0.0001)
disc_optimizer = optim.Adam(discriminator.parameters(), lr=0.0001)


In [156]:
# Generate a random permutation of the class labels
random_indices = torch.randperm(num_classes)

# Create a tensor of random class labels
random_class_labels = torch.tensor([class_labels[i] for i in random_indices])


# random_indices = torch.randint(0, num_classes-1, (batch_size, 1))
# labels = random_class_labels[random_indices] # generate random labels

In [ ]:
random_indices = torch.randint(0, len(class_labels)-1, (batch_size, 1))
labels = random_class_labels[random_indices] # generate random labels

In [157]:
train_data

In [ ]:
# random_indices = torch.randint(0, len(class_labels)-1, (batch_size, 1))
# labels = random_class_labels[random_indices] # generate random labels
# labels = torch.randint(0, num_classes, (batch_size, 1))
# fake_data = fake_data.view(-1,11)

In [205]:
# Train the CGAN
for epoch in range(num_epochs):
    for i, (data, label) in enumerate(train_data):
        # Generate fake data with labels
        noise = torch.randn(batch_size, 11)
        if data.size()[0] < batch_size:
            continue
        fake_data = generator(torch.cat((noise, label), 1)) # concatenate noise and labels as input to generato

        # Train the discriminator
        disc_optimizer.zero_grad()
        real_loss = criterion(discriminator(data), torch.ones(batch_size, 1))
        fake_loss = criterion(discriminator(fake_data), torch.zeros(batch_size, 1))
        disc_loss = real_loss + fake_loss
        disc_loss.backward(retain_graph=True)
        disc_optimizer.step()

        # Train the generator
        gen_optimizer.zero_grad()
        gen_loss = criterion(discriminator(fake_data), torch.ones(batch_size, 1))
        gen_loss.backward(retain_graph=True)
        """retain_graph tells the autograd engine to retain the intermediate values of the graph,
        instead of freeing them, so that they can be used in the next backward pass."""
        gen_optimizer.step()

    # predictions = discriminator(data)
    # print("Predictions for real data: ", predictions, "with label: ", label)
    # predictions = discriminator(fake_data)
    # print("Predictions for fake data: ", predictions, "with label: ", label)
    print(f"Epoch {epoch + 1}: ")
    print(f"Generator Loss: {gen_loss}")
    print(f"Discriminator Loss : {disc_loss}")

Epoch 1: 
Generator Loss: 18.613555908203125
Discriminator Loss : 100.0
Epoch 2: 
Generator Loss: 18.71721839904785
Discriminator Loss : 100.0
Epoch 3: 
Generator Loss: 18.820920944213867
Discriminator Loss : 100.0
Epoch 4: 
Generator Loss: 18.92326545715332
Discriminator Loss : 100.0
Epoch 5: 
Generator Loss: 19.025487899780273
Discriminator Loss : 100.0
Epoch 6: 
Generator Loss: 19.126768112182617
Discriminator Loss : 100.0
Epoch 7: 
Generator Loss: 19.228456497192383
Discriminator Loss : 100.0
Epoch 8: 
Generator Loss: 19.3303279876709
Discriminator Loss : 100.0
Epoch 9: 
Generator Loss: 19.43107795715332
Discriminator Loss : 100.0
Epoch 10: 
Generator Loss: 19.533185958862305
Discriminator Loss : 100.0
Epoch 11: 
Generator Loss: 19.633554458618164
Discriminator Loss : 100.0
Epoch 12: 
Generator Loss: 19.733858108520508
Discriminator Loss : 100.0
Epoch 13: 
Generator Loss: 19.834327697753906
Discriminator Loss : 100.0
Epoch 14: 
Generator Loss: 19.934541702270508
Discriminator Loss 

KeyboardInterrupt: 

In [ ]:
# Generate new data using the trained generator with a specific label
label = 3  # specify the label you want the data to be generated for
noise = torch.randn(batch_size, 11)
label_tensor = torch.ones(batch_size, 1) * label  # create a tensor with the specified label
generated_data = generator(torch.cat((noise, label_tensor), 1))

# Print the generated data
print(generated_data)

In [ ]:
# Assign the GPU device to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the models and tensors to the GPU
generator.to(device)
discriminator.to(device)
criterion.to(device)

for epoch in range(num_epochs):
    for i, data in enumerate(train_data):
        # Move the data tensor to the GPU
        data = data.to(device)
        # Generate fake data with labels
        noise = torch.randn(batch_size, 11).to(device)
        labels = torch.randint(0, num_classes, (batch_size, 1)).to(device)  # generate random labels
        fake_data = generator(torch.cat((noise, labels), 1))  # concatenate noise and labels as input to generator
        .
        .
        .


In [ ]:
# Define the list of class labels
class_labels = [3, 6, 8, 12, 15, 20]
num_classes = len(class_labels)

# Generate a random permutation of the class labels
random_indices = torch.randperm(num_classes)

# Create a tensor of random class labels
random_class_labels = torch.tensor([class_labels[i] for i in random_indices])

for epoch in range(num_epochs):
    for i, data in enumerate(train_data):
        # Generate random class labels
        random_indices = torch.randint(0, num_classes, (batch_size, 1))
        labels = random_class_labels[random_indices]
        .
        .
        .


In [ ]:
for i, (data, label) in enumerate(train_data):
    # Generate fake data with labels
    noise = torch.randn(batch_size, 11)
    fake_data = generator(torch.cat((noise, label), 1))
    predictions = discriminator(data)
    print("Predictions for real data: ", predictions)
    predictions = discriminator(fake_data)
    print("Predictions for fake data: ", predictions)


for i, (data, label) in enumerate(train_data):
    # Generate fake data with labels
    noise = torch.randn(batch_size, 11)
    fake_data = generator(torch.cat((noise, label), 1))
    predictions = discriminator(data)
    print("Predictions for real data: ", predictions, "with label: ", label)
    predictions = discriminator(fake_data)
    print("Predictions for fake data: ", predictions, "with label: ", label)
